# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [3]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-24f4392fa4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Production company explorative search") 

Consider the following exploratory information need:

> investigate the main companies working on cinema-related content. We want to know which are the main television production companies and the film production companies, which company distributes more film and some consideration about awards.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Netflix

3. Identify the BGP for television production company

4. Identify the BGP for film production company

5. Find the top-5 companies (amongst television production company and film production company) which produced the highest number of crime films (the result set must be a list of triples company IRI, label and #crime film).

6. Find the company (amongst television production company and film production company) which distributed more films (of any kind of film genre) that they did not produce (the result set must be a list of triples company IRI, label and #film).

7. Find how many company are listed in every stock exchange market (the result set must be a list of triples stock exchange IRI, label and #companies). 

8. Identify the BGP for Academy Award

9. Find the companies (among television and film production companies) that won at least 5 Academy Awards for Best Actress for the movies they produced (the result set must be a list of triples company IRI, label and #awards).

## Task 1
Identify the BGP for films

In [4]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [48]:
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    wd:Q36479 wdt:P136 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2143665'), ('name', "children's film")]
[('p', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama')]
[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
[('p', 'http://www.wikidata.org/entity/Q28968511'), ('name', 'traditionally animated film')]
4


Final query for this task

In [52]:
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
     wd:Q842256 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [54]:
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
     wd:Q842256 wdt:P279 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


In [55]:
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
     wd:Q11424 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p',

In [5]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
queryString = """
SELECT DISTINCT ?p ?sname ?obj ?oname
WHERE {
   # bind something
     wd:Q11424 ?p ?obj.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('sname', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q1660187'), ('oname', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P345'), ('oname', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1040'), ('oname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P344'), ('oname', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('sname', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q17738'), ('oname', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('sname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q55321'), ('oname', 'Ca

## Task 2
Identify the BGP for Netflix

In [13]:
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
#[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
     ?p wdt:P227 ?o.
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name, "Netflix"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
1


In [16]:
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
#[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
#[('p', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     wd:Q907311 ?p ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('is', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q10689397'), ('of', 'television production company')]
[('p', 'http://www.wikidata.org/prop/direct/P400'), ('is', 'platform'), ('o', 'http://www.wikidata.org/entity/Q23930024'), ('of', 'streaming media receiver')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('is', 'award received'), ('o', 'http://www.wikidata.org/entity/Q838121'), ('of', 'Peabody Awards')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('is', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q849363'), ('of', 'The Vanguard Group')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('is', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q220499'), ('of', 'streaming media')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('is', 'industry'), ('o', 'http://www.wikidata.org/entity/Q220499'), ('of', 'streaming media')]
[('p', 'http://www.wikidata.org/prop/direct/P4

Final query for this task

In [17]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
#[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
#[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
#[('p', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     wd:Q907311 ?p ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('is', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q10689397'), ('of', 'television production company')]
[('p', 'http://www.wikidata.org/prop/direct/P400'), ('is', 'platform'), ('o', 'http://www.wikidata.org/entity/Q23930024'), ('of', 'streaming media receiver')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('is', 'award received'), ('o', 'http://www.wikidata.org/entity/Q838121'), ('of', 'Peabody Awards')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('is', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q849363'), ('of', 'The Vanguard Group')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('is', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q220499'), ('of', 'streaming media')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('is', 'industry'), ('o', 'http://www.wikidata.org/entity/Q220499'), ('of', 'streaming media')]
[('p', 'http://www.wikidata.org/prop/direct/P4

## Task 3
Identify the BGP for television production company

In [50]:
# write your queries
#('o', 'http://www.wikidata.org/entity/Q10689397'), ('of', 'television production company')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     wd:Q10689397 ?p ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1762059'), ('of', 'film production company')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q19364326'), ('of', 'media studio facility')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('is', 'industry'), ('o', 'http://www.wikidata.org/entity/Q1407228'), ('of', 'television production')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q11396960'), ('of', 'production company')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('is', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8836004'), ('of', 'Category:Television production companies')]
5


Final query for this task

In [ ]:
# write your final query
#('o', 'http://www.wikidata.org/entity/Q10689397'), ('of', 'television production company')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     wd:Q10689397 ?p ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

## Task 4
Identify the BGP for film production company

In [19]:
# write your queries
# ('o', 'http://www.wikidata.org/entity/Q1762059'), ('of', 'film production company')]
#('o', 'http://www.wikidata.org/entity/Q10689397'), ('of', 'television production company')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     wd:Q1762059 ?p ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('is', 'different from'), ('o', 'http://www.wikidata.org/entity/Q375336'), ('of', 'film studio')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('is', 'industry'), ('o', 'http://www.wikidata.org/entity/Q1415395'), ('of', 'film industry')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q104649845'), ('of', 'film organization')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('is', 'part of'), ('o', 'http://www.wikidata.org/entity/Q625568'), ('of', 'show business')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q11396960'), ('of', 'production company')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('is', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6483398'), ('of', 'Category:Film production companies')]
6


Final query for this task

In [33]:
# write your final query
# ('o', 'http://www.wikidata.org/entity/Q1762059'), ('of', 'film production company')]
#('o', 'http://www.wikidata.org/entity/Q10689397'), ('of', 'television production company')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     wd:Q1762059 ?p ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('is', 'different from'), ('o', 'http://www.wikidata.org/entity/Q375336'), ('of', 'film studio')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('is', 'industry'), ('o', 'http://www.wikidata.org/entity/Q1415395'), ('of', 'film industry')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q104649845'), ('of', 'film organization')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('is', 'part of'), ('o', 'http://www.wikidata.org/entity/Q625568'), ('of', 'show business')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q11396960'), ('of', 'production company')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('is', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6483398'), ('of', 'Category:Film production companies')]
6


## Task 5
Find the top-5 companies (amongst television production company and film production company) which produced the highest number of crime films (the result set must be a list of triples company IRI, label and #crime film).

In [193]:
# Using Film BGP to get Crime Films
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
queryString = """
SELECT DISTINCT ?p ?sname ?obj ?oname
WHERE {
   # bind something
     ?p wdt:P136 ?obj.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
   FILTER(REGEX(?oname,"crime film"^^xsd:string ))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q13693967'), ('sname', 'Cross'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film')]
[('p', 'http://www.wikidata.org/entity/Q20724304'), ('sname', 'The Purge: Election Year'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film')]
[('p', 'http://www.wikidata.org/entity/Q15865603'), ('sname', 'After Dark'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film')]
[('p', 'http://www.wikidata.org/entity/Q1755064'), ('sname', "Sharky's Machine"), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film')]
[('p', 'http://www.wikidata.org/entity/Q11510316'), ('sname', 'Love Under Fire'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film')]
[('p', 'http://www.wikidata.org/entity/Q16020981'), ('sname', 'The Amazing Partnership'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film')]
[('p', 'http://www.wikidata.org/entity/Q16

In [196]:
# Finding names of production company using Query 9 
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
queryString = """
SELECT DISTINCT ?p ?sname ?obj ?oname ?Pname ?pcoy
WHERE {
   # bind something
     ?p wdt:P136 ?obj;
        wdt:P272 ?Pname.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
   ?Pname sc:name ?pcoy
   FILTER(REGEX(?oname,"crime film"^^xsd:string ))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q26266013'), ('sname', 'Tatort: Die Kunst des Krieges'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film'), ('Pname', 'http://www.wikidata.org/entity/Q2367528'), ('pcoy', 'Superfilm')]
[('p', 'http://www.wikidata.org/entity/Q459057'), ('sname', 'Monster'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film'), ('Pname', 'http://www.wikidata.org/entity/Q5204823'), ('pcoy', 'DEJ Productions')]
[('p', 'http://www.wikidata.org/entity/Q496113'), ('sname', 'City of Industry'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film'), ('Pname', 'http://www.wikidata.org/entity/Q17346576'), ('pcoy', 'Largo Entertainment')]
[('p', 'http://www.wikidata.org/entity/Q498983'), ('sname', 'Young and Innocent'), ('obj', 'http://www.wikidata.org/entity/Q959790'), ('oname', 'crime film'), ('Pname', 'http://www.wikidata.org/entity/Q2756503'), ('pcoy', 'Gaumont-British Picture Corporation')]
[('p'

In [197]:
# Counting number of films for each company
#[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
#[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfFilms) ?Pname ?pcoy
WHERE {
   # bind something
     ?p wdt:P136 ?obj;
        wdt:P272 ?Pname.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
   ?Pname sc:name ?pcoy
   FILTER(REGEX(?oname,"crime film"^^xsd:string ))
}
GROUP BY ?pcoy ?Pname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '2'), ('Pname', 'http://www.wikidata.org/entity/Q47153414'), ('pcoy', 'Athos Films')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q108183211'), ('pcoy', 'Lumen Films')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q17377479'), ('pcoy', 'Edward Small Productions')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q605307'), ('pcoy', 'Les films du losange')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q624886'), ('pcoy', 'FilmDistrict')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q82110'), ('pcoy', 'Steven Seagal')]
[('NumOfFilms', '2'), ('Pname', 'http://www.wikidata.org/entity/Q4341103'), ('pcoy', 'Profit')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q2748604'), ('pcoy', 'Film i Väst')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/entity/Q7127609'), ('pcoy', 'Palisades Tartan')]
[('NumOfFilms', '1'), ('Pname', 'http://www.wikidata.org/e

Final query for this task

In [200]:
# write your final query
# Organising in Descending order and selecting top 5 Companies
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfFilms) ?Pname ?pcoy
WHERE {
   # bind something
     ?p wdt:P136 ?obj;
        wdt:P272 ?Pname.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
   ?Pname sc:name ?pcoy
   FILTER(REGEX(?oname,"crime film"^^xsd:string ))
}
GROUP BY ?pcoy ?Pname
ORDER BY DESC(?NumOfFilms)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '161'), ('Pname', 'http://www.wikidata.org/entity/Q126399'), ('pcoy', 'Warner Bros.')]
[('NumOfFilms', '118'), ('Pname', 'http://www.wikidata.org/entity/Q186941'), ('pcoy', 'Columbia Pictures')]
[('NumOfFilms', '113'), ('Pname', 'http://www.wikidata.org/entity/Q168383'), ('pcoy', 'Universal Pictures')]
[('NumOfFilms', '112'), ('Pname', 'http://www.wikidata.org/entity/Q179200'), ('pcoy', 'Metro-Goldwyn-Mayer')]
[('NumOfFilms', '109'), ('Pname', 'http://www.wikidata.org/entity/Q159846'), ('pcoy', 'Paramount Pictures')]
5


## Task 6
Find the company (amongst television production company and film production company) which distributed more films (of any kind of film genre) that they did not produce (the result set must be a list of triples company IRI, label and #film).

In [205]:
# Using Film BGP to get ID for Distributors
queryString = """
SELECT DISTINCT ?p ?sname ?obj ?oname
WHERE {
   # bind something
     wd:Q11424 ?p ?obj.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
   FILTER(REGEX(?oname,"distributed"^^xsd:string))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P750'), ('oname', 'distributed by')]
1


In [210]:
#getting all the production companies which distributed films
queryString = """
SELECT DISTINCT ?p ?sname ?obj ?oname
WHERE {
   # bind something
     ?p wdt:P750 ?obj.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q10868675'), ('sname', 'Kakyuusei 2'), ('obj', 'http://www.wikidata.org/entity/Q5366830'), ('oname', 'DMM.com')]
[('p', 'http://www.wikidata.org/entity/Q8141093'), ('sname', 'Elf-ban Kakyūsei'), ('obj', 'http://www.wikidata.org/entity/Q5366830'), ('oname', 'DMM.com')]
[('p', 'http://www.wikidata.org/entity/Q15863744'), ('sname', 'Thegidi'), ('obj', 'http://www.wikidata.org/entity/Q7785300'), ('oname', 'C. V. Kumar')]
[('p', 'http://www.wikidata.org/entity/Q17060192'), ('sname', 'Kimmy Dora: Ang Kiyemeng Prequel'), ('obj', 'http://www.wikidata.org/entity/Q17088846'), ('oname', 'Spring Films')]
[('p', 'http://www.wikidata.org/entity/Q17063445'), ('sname', '1'), ('obj', 'http://www.wikidata.org/entity/Q17089396'), ('oname', 'Exclusive Media')]
[('p', 'http://www.wikidata.org/entity/Q17074219'), ('sname', 'Serndhu Polama'), ('obj', 'http://www.wikidata.org/entity/Q23023125'), ('oname', 'Dream Factory')]
[('p', 'http://www.wikidata.org/entity/Q

In [218]:
#Counting Total Films Distributed by Each company
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfFilms) ?obj ?oname
WHERE {
   # bind something
     ?p wdt:P750 ?obj.
   # get the label
   ?obj sc:name ?oname.
}
GROUP BY ?oname ?obj
ORDER BY DESC(?NumOfFilms)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '26267'), ('obj', 'http://www.wikidata.org/entity/Q907311'), ('oname', 'Netflix')]
[('NumOfFilms', '8865'), ('obj', 'http://www.wikidata.org/entity/Q337535'), ('oname', 'Steam')]
[('NumOfFilms', '5794'), ('obj', 'http://www.wikidata.org/entity/Q80948336'), ('oname', 'FandangoNow')]
[('NumOfFilms', '3535'), ('obj', 'http://www.wikidata.org/entity/Q9589'), ('oname', 'iTunes')]
[('NumOfFilms', '3525'), ('obj', 'http://www.wikidata.org/entity/Q3070866'), ('oname', 'Nintendo eShop')]
[('NumOfFilms', '3392'), ('obj', 'http://www.wikidata.org/entity/Q42328566'), ('oname', 'Humble Store')]
[('NumOfFilms', '3267'), ('obj', 'http://www.wikidata.org/entity/Q159846'), ('oname', 'Paramount Pictures')]
[('NumOfFilms', '3056'), ('obj', 'http://www.wikidata.org/entity/Q368215'), ('oname', 'App Store')]
[('NumOfFilms', '2785'), ('obj', 'http://www.wikidata.org/entity/Q126399'), ('oname', 'Warner Bros.')]
[('NumOfFilms', '2777'), ('obj', 'http://www.wikidata.org/entity/Q5371838')

In [263]:
#excluding movies produced by these companies
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfFilms) ?obj ?oname
WHERE {
   # bind something
     ?p wdt:P750 ?obj.
   # get the label
   ?obj sc:name ?oname.
   
   FILTER NOT EXISTS {?p wdt:P272 ?op}
}
GROUP BY ?oname ?obj
ORDER BY DESC(?NumOfFilms)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfFilms', '14965'), ('obj', 'http://www.wikidata.org/entity/Q907311'), ('oname', 'Netflix')]
[('NumOfFilms', '8855'), ('obj', 'http://www.wikidata.org/entity/Q337535'), ('oname', 'Steam')]
[('NumOfFilms', '3524'), ('obj', 'http://www.wikidata.org/entity/Q3070866'), ('oname', 'Nintendo eShop')]
[('NumOfFilms', '3387'), ('obj', 'http://www.wikidata.org/entity/Q42328566'), ('oname', 'Humble Store')]
[('NumOfFilms', '3346'), ('obj', 'http://www.wikidata.org/entity/Q9589'), ('oname', 'iTunes')]
[('NumOfFilms', '3045'), ('obj', 'http://www.wikidata.org/entity/Q368215'), ('oname', 'App Store')]
[('NumOfFilms', '2264'), ('obj', 'http://www.wikidata.org/entity/Q1486288'), ('oname', 'GOG.com')]
[('NumOfFilms', '2245'), ('obj', 'http://www.wikidata.org/entity/Q5371838'), ('oname', 'Vudu')]
[('NumOfFilms', '2241'), ('obj', 'http://www.wikidata.org/entity/Q79576'), ('oname', 'Google Play')]
[('NumOfFilms', '1775'), ('obj', 'http://www.wikidata.org/entity/Q159846'), ('oname', 'Paramoun

Final query for this task

In [ ]:
# write your final query
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfFilms) ?obj ?oname
WHERE {
   # bind something
     ?p wdt:P750 ?obj.
   # get the label
   ?obj sc:name ?oname.
   
   FILTER NOT EXISTS {?p wdt:P272 ?op}
}
GROUP BY ?oname ?obj
ORDER BY DESC(?NumOfFilms)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

## Task 7
Find how many company are listed in every stock exchange market (the result set must be a list of triples stock exchange IRI, label and #companies).

In [240]:
# Exploring Production Company for finding stock exchange, Using IRI from BGP of film production 
# ('o', 'http://www.wikidata.org/entity/Q11396960'), ('of', 'production company')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
      wd:Q11396960 ?p ?o .
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('is', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q21198548'), ('of', 'audiovisual production')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1331793'), ('of', 'media company')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q783794'), ('of', 'company')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('is', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P272'), ('of', 'production company')]
4


In [241]:
# Exploring Media Company for finding stock exchange 
# ('o', 'http://www.wikidata.org/entity/Q1331793'), ('of', 'media company')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
      wd:Q1331793 ?p ?o .
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('is', 'different from'), ('o', 'http://www.wikidata.org/entity/Q19364326'), ('of', 'media studio facility')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4830453'), ('of', 'business')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('is', 'uses'), ('o', 'http://www.wikidata.org/entity/Q11024'), ('of', 'communication')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('is', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q11033'), ('of', 'mass media')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('is', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q340169'), ('of', 'communication medium')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('is', 'uses'), ('o', 'http://www.wikidata.org/entity/Q340169'), ('of', 'communication medium')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('is', 'subclass of'), ('o', 'http:

In [243]:
# Exploring Business
#('o', 'http://www.wikidata.org/entity/Q4830453'), ('of', 'business')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
      wd:Q4830453 ?p ?o .
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('is', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P414'), ('of', 'stock exchange')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('is', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2002'), ('of', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('is', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1278'), ('of', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('is', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1320'), ('of', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('is', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1056'), ('of', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('is', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1329'), ('of', 'phone numbe

In [244]:
#Exploring Stock Exchange
#('o', 'http://www.wikidata.org/entity/P414'), ('of', 'stock exchange')]
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
   # bind something
     ?p wdt:P414 ?o .
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
}
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q53247'), ('is', 'Panasonic Corporation'), ('o', 'http://www.wikidata.org/entity/Q1526647'), ('of', 'Philippine Stock Exchange')]
[('p', 'http://www.wikidata.org/entity/Q444971'), ('is', 'Altri'), ('o', 'http://www.wikidata.org/entity/Q2415561'), ('of', 'Euronext Lisbon')]
[('p', 'http://www.wikidata.org/entity/Q10366645'), ('is', 'Stoxx Europe 600 Index'), ('o', 'http://www.wikidata.org/entity/Q2415561'), ('of', 'Euronext Lisbon')]
[('p', 'http://www.wikidata.org/entity/Q981010'), ('is', 'EURO STOXX 50'), ('o', 'http://www.wikidata.org/entity/Q2415561'), ('of', 'Euronext Lisbon')]
[('p', 'http://www.wikidata.org/entity/Q2044697'), ('is', 'PSI-20'), ('o', 'http://www.wikidata.org/entity/Q2415561'), ('of', 'Euronext Lisbon')]
[('p', 'http://www.wikidata.org/entity/Q1341222'), ('is', 'Energias de Portugal'), ('o', 'http://www.wikidata.org/entity/Q2415561'), ('of', 'Euronext Lisbon')]
[('p', 'http://www.wikidata.org/entity/Q450420'), ('is', '

In [245]:
#Grouping by Stock Exchange and counting number of companies in each exchange
queryString = """
SELECT DISTINCT (COUNT(?p) as ?NumOfCoys) ?o ?ex
WHERE {
   # bind something
     ?p wdt:P414 ?o .
   # get the label
   ?o sc:name ?ex
}
GROUP BY ?ex ?o
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('NumOfCoys', '40'), ('o', 'http://www.wikidata.org/entity/Q661834'), ('ex', 'SIX Swiss Exchange')]
[('NumOfCoys', '6'), ('o', 'http://www.wikidata.org/entity/Q819468'), ('ex', 'Xetra')]
[('NumOfCoys', '460'), ('o', 'http://www.wikidata.org/entity/Q217475'), ('ex', 'Tokyo Stock Exchange')]
[('NumOfCoys', '1'), ('o', 'http://www.wikidata.org/entity/Q4890307'), ('ex', 'Bilbao Stock Exchange')]
[('NumOfCoys', '11'), ('o', 'http://www.wikidata.org/entity/Q1984912'), ('ex', 'PFTS Ukraine Stock Exchange')]
[('NumOfCoys', '346'), ('o', 'http://www.wikidata.org/entity/Q6529599'), ('ex', 'Nepal Stock Exchange')]
[('NumOfCoys', '58'), ('o', 'http://www.wikidata.org/entity/Q581755'), ('ex', 'Nasdaq Helsinki')]
[('NumOfCoys', '3'), ('o', 'http://www.wikidata.org/entity/Q1003968'), ('ex', 'Bulgarian Stock Exchange')]
[('NumOfCoys', '1'), ('o', 'http://www.wikidata.org/entity/Q22907145'), ('ex', 'CEX.io')]
[('NumOfCoys', '2'), ('o', 'http://www.wikidata.org/entity/Q2718247'), ('ex', 'Eurone

Final query for this task

In [246]:
# write your final query
queryString = """
SELECT DISTINCT (COUNT(?p) as ?NumOfCoys) ?o ?ex
WHERE {
   # bind something
     ?p wdt:P414 ?o .
   # get the label
   ?o sc:name ?ex
}
GROUP BY ?ex ?o
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('NumOfCoys', '40'), ('o', 'http://www.wikidata.org/entity/Q661834'), ('ex', 'SIX Swiss Exchange')]
[('NumOfCoys', '6'), ('o', 'http://www.wikidata.org/entity/Q819468'), ('ex', 'Xetra')]
[('NumOfCoys', '460'), ('o', 'http://www.wikidata.org/entity/Q217475'), ('ex', 'Tokyo Stock Exchange')]
[('NumOfCoys', '1'), ('o', 'http://www.wikidata.org/entity/Q4890307'), ('ex', 'Bilbao Stock Exchange')]
[('NumOfCoys', '11'), ('o', 'http://www.wikidata.org/entity/Q1984912'), ('ex', 'PFTS Ukraine Stock Exchange')]
[('NumOfCoys', '346'), ('o', 'http://www.wikidata.org/entity/Q6529599'), ('ex', 'Nepal Stock Exchange')]
[('NumOfCoys', '58'), ('o', 'http://www.wikidata.org/entity/Q581755'), ('ex', 'Nasdaq Helsinki')]
[('NumOfCoys', '3'), ('o', 'http://www.wikidata.org/entity/Q1003968'), ('ex', 'Bulgarian Stock Exchange')]
[('NumOfCoys', '1'), ('o', 'http://www.wikidata.org/entity/Q22907145'), ('ex', 'CEX.io')]
[('NumOfCoys', '2'), ('o', 'http://www.wikidata.org/entity/Q2718247'), ('ex', 'Eurone

## Task 8
Identify the BGP for Academy Award

In [14]:
# write your queries
# from Task 1 BGP for Films
queryString = """
SELECT DISTINCT ?p ?sname ?obj ?oname
WHERE {{
   # bind something
     wd:Q11424 ?p ?obj.
   # get the label
   ?p sc:name ?sname.
   ?obj sc:name ?oname.
   }
   Union
   {
    ?p ?obj wd:Q11424.
    ?p sc:name ?sname.
   ?obj sc:name ?oname.
    
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('sname', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q1660187'), ('oname', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P345'), ('oname', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1040'), ('oname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('sname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P344'), ('oname', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('sname', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q17738'), ('oname', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('sname', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q55321'), ('oname', 'Ca

In [17]:
#('obj', 'http://www.wikidata.org/entity/P166'), ('oname', 'award received')]
# ID for Award Recieved and further discover
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
      ?n wdt:P166 ?obj.
   # get the label
   ?obj sc:name ?name.
   FILTER(REGEX(?name, "Academy Award"^^xsd:string))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q17001892'), ('name', 'Africa Movie Academy Award for Best Actress in a Supporting Role')]
[('obj', 'http://www.wikidata.org/entity/Q17001922'), ('name', 'Africa Movie Academy Award for Most Promising Actor')]
[('obj', 'http://www.wikidata.org/entity/Q148609'), ('name', '16th Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q655089'), ('name', 'International Indian Film Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q918125'), ('name', '45th Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q22235305'), ('name', 'Academy Award for Best Original Musical or Comedy Score')]
[('obj', 'http://www.wikidata.org/entity/Q547575'), ('name', 'Polish Academy Award for Best Costume Design')]
[('obj', 'http://www.wikidata.org/entity/Q18204254'), ('name', 'Africa Movie Academy Award for Best Cinematography')]
[('obj', 'http://www.wikidata.org/entity/Q375039'), ('name', 'Student Academy Awards')]
[('obj', 'http://www.wikid

In [18]:
#Getiing ID for 16th Academy Award
#[('obj', 'http://www.wikidata.org/entity/Q148609'), ('name', '16th Academy Awards')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   {
   # bind something
      wd:Q148609 ?n ?obj.
   # get the label
   ?obj sc:name ?name.
   }
   UNION
   {
   # bind something
    ?obj ?n wd:Q148609.
   # get the label
   ?obj sc:name ?name.
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q148732'), ('name', 'Jack Benny')]
[('obj', 'http://www.wikidata.org/entity/Q508528'), ('name', '15th Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q16913666'), ('name', 'Academy Awards ceremony')]
[('obj', 'http://www.wikidata.org/entity/Q917159'), ('name', '17th Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q1122565'), ('name', "Grauman's Chinese Theatre")]
[('obj', 'http://www.wikidata.org/entity/Q504191'), ('name', 'W. Howard Greene')]
[('obj', 'http://www.wikidata.org/entity/Q708870'), ('name', 'Arthur Charles Miller')]
[('obj', 'http://www.wikidata.org/entity/Q8056460'), ('name', "You'll Never Know")]
9


Final query for this task

In [27]:
# write your final query
#Using ID for Academy Awards
#[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
     wd:Q19020 ?obj ?p.
   # get the label
   ?obj sc:name ?name.
   
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/prop/direct/P1407'), ('name', 'MusicBrainz series ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('obj', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('obj', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('obj', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('obj', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('obj', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('obj', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('obj', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('obj', 'http://www.wikida

## Task 9
Find the companies (among television and film production companies) that won at least 5 Academy Awards for Best Actress for the movies they produced (the result set must be a list of triples company IRI, label and #awards).

In [36]:
# Using Category for reciepts of this award ID from task 8
#[('obj', 'http://www.wikidata.org/prop/direct/P2517'), ('name', 'category for recipients of this award')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
      wd:Q19020 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('obj', 'http://www.wikidata.org/entity/Q103618'), ('name', 'Academy Award for Best Actress')]
[('obj', 'http://www.wikidata.org/entity/Q106301'), ('name', 'Academy Award for Best Supporting Actress')]
[('obj', 'http://www.wikidata.org/entity/Q107258'), ('name', 'Academy Award for Best Writing, Adapted Screenplay')]
[('obj', 'http://www.wikidata.org/entity/Q112243'), ('name', 'Academy Award for Best Original Song')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q41417'), ('name', 'Academy Award for Best Writing, Original Screenplay')]
[('obj', 'http://www.wikidata.org/entity/Q103916'), ('name', 'Academy Award for Best Actor')]
[('obj', 'http://www.wikidata.org/entity/Q105304'), ('name', 'Academy Award for Be

In [59]:
#Using ID for Best Actress Award
#[('obj', 'http://www.wikidata.org/entity/Q103618'), ('name', 'Academy Award for Best Actress')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
     ?p ?obj  wd:Q103618.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('obj', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('obj', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('obj', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('obj', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('obj', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
6


In [72]:
#Using ID for Award Recieved by the movies / Shows for best actress
#[('obj', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
     ?obj wdt:P166 wd:Q103618.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q12003'), ('name', 'Cher')]
[('obj', 'http://www.wikidata.org/entity/Q43252'), ('name', 'Sophia Loren')]
[('obj', 'http://www.wikidata.org/entity/Q451603'), ('name', 'On Golden Pond')]
[('obj', 'http://www.wikidata.org/entity/Q459057'), ('name', 'Monster')]
[('obj', 'http://www.wikidata.org/entity/Q187033'), ('name', 'Sally Field')]
[('obj', 'http://www.wikidata.org/entity/Q40523'), ('name', 'Julia Roberts')]
[('obj', 'http://www.wikidata.org/entity/Q44063'), ('name', 'Reese Witherspoon')]
[('obj', 'http://www.wikidata.org/entity/Q7088045'), ('name', 'Olivia Colman')]
[('obj', 'http://www.wikidata.org/entity/Q253282'), ('name', 'Marie Dressler')]
[('obj', 'http://www.wikidata.org/entity/Q652069'), ('name', 'Butterfield 8')]
[('obj', 'http://www.wikidata.org/entity/Q1737875'), ('name', 'Come Back, Little Sheba')]
[('obj', 'http://www.wikidata.org/entity/Q29328'), ('name', 'Brie Larson')]
[('obj', 'http://www.wikidata.org/entity/Q102642'),

In [122]:
# Combining Films BGP for filtering out the names of films awarded with Best Actress Award
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
  {
  # bind something
      ?p wdt:P31 wd:Q11424;
          wdt:P166 ?o.
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of
   FILTER(REGEX(?of,"Academy Award for Best Actress"^^xsd:string))
   }
   UNION
   {
       
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q451603'), ('is', 'On Golden Pond'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('of', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/entity/Q459057'), ('is', 'Monster'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('of', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/entity/Q652069'), ('is', 'Butterfield 8'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('of', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/entity/Q1737875'), ('is', 'Come Back, Little Sheba'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('of', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/entity/Q133654'), ('is', 'The Silence of the Lambs'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('of', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/entity/Q159063'), ('is', 'The Reader'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('of', 'Academy Award fo

In [121]:
#Exploring Single Movie for Getting its Production Company
#[('p', 'http://www.wikidata.org/entity/Q233464'), ('is', 'Annie Hall')
queryString = """
SELECT DISTINCT ?p ?is ?o ?of
WHERE {
  # bind something
     wd:Q117315 wdt:P272 ?o .
          
   # get the label
   #?p sc:name ?is.
   ?o sc:name ?of.
   #FILTER(REGEX(?o, "production company"^^xsd:string))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4036210'), ('of', 'Ciby 2000')]
[('o', 'http://www.wikidata.org/entity/Q4824145'), ('of', 'Australian Film Commission')]
[('o', 'http://www.wikidata.org/entity/Q17335626'), ('of', 'Jan Chapman Productions')]
3


In [174]:
# Combining Films BGP for filtering out the names of films awarded with Best Actress Award
queryString = """
SELECT DISTINCT (COUNT (?is) as ?Awards) ?gh
WHERE {
  {
  # bind something
      ?p wdt:P31 wd:Q11424;
          wdt:P166 ?o;
          wdt:P272 ?m.
          
        #?m wdt:P31 ?hi.
        
          
      
   # get the label
   ?p sc:name ?is.
   ?o sc:name ?of.
    ?m sc:name ?gh.
   #?hi sc:name ?ku.
    
   
   FILTER(REGEX(?of,"Academy Award for Best Actress"^^xsd:string))
   #FILTER(REGEX(?ku,"film production company"^^xsd:string))
   #FILTER(REGEX(?ku,"television production company"^^xsd:string))
   }
}
GROUP BY ?gh
ORDER BY DESC(?Awards)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Awards', '11'), ('gh', 'Paramount Pictures')]
[('Awards', '11'), ('gh', 'Metro-Goldwyn-Mayer')]
[('Awards', '9'), ('gh', 'Warner Bros.')]
[('Awards', '5'), ('gh', '20th Century Studios')]
[('Awards', '4'), ('gh', 'Columbia Pictures')]
[('Awards', '4'), ('gh', 'RKO Pictures')]
[('Awards', '3'), ('gh', 'Film4 Productions')]
[('Awards', '2'), ('gh', 'United Artists Corporation')]
[('Awards', '2'), ('gh', 'ITC Entertainment')]
[('Awards', '2'), ('gh', 'Embassy Pictures')]
[('Awards', '2'), ('gh', 'Working Title Films')]
[('Awards', '2'), ('gh', 'Killer Films')]
[('Awards', '2'), ('gh', 'The Weinstein Company')]
[('Awards', '2'), ('gh', 'Searchlight Pictures')]
[('Awards', '2'), ('gh', 'Orion Pictures')]
[('Awards', '2'), ('gh', 'Pathé')]
[('Awards', '2'), ('gh', 'Lionsgate')]
[('Awards', '2'), ('gh', 'Universal Pictures')]
[('Awards', '2'), ('gh', 'PolyGram Filmed Entertainment')]
[('Awards', '1'), ('gh', 'Jan Chapman Productions')]
[('Awards', '1'), ('gh', 'Element Pictures')]


Final query for this task

In [178]:
# write your final query
queryString = """
SELECT DISTINCT (COUNT (?is) as ?Awards) ?gh as ?ProductionCompanies ?m as ?IRI
WHERE {
  {
  # bind something
      ?p wdt:P31 wd:Q11424;
          wdt:P166 ?o;
          wdt:P272 ?m.
          
   # get the label
       ?p sc:name ?is.
       ?o sc:name ?of.
       ?m sc:name ?gh.
    
   
   FILTER(REGEX(?of,"Academy Award for Best Actress"^^xsd:string))
   }
}
GROUP BY ?gh ?m
ORDER BY DESC(?Awards)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Awards', '11'), ('ProductionCompanies', 'Paramount Pictures'), ('IRI', 'http://www.wikidata.org/entity/Q159846')]
[('Awards', '11'), ('ProductionCompanies', 'Metro-Goldwyn-Mayer'), ('IRI', 'http://www.wikidata.org/entity/Q179200')]
[('Awards', '9'), ('ProductionCompanies', 'Warner Bros.'), ('IRI', 'http://www.wikidata.org/entity/Q126399')]
[('Awards', '5'), ('ProductionCompanies', '20th Century Studios'), ('IRI', 'http://www.wikidata.org/entity/Q434841')]
[('Awards', '4'), ('ProductionCompanies', 'RKO Pictures'), ('IRI', 'http://www.wikidata.org/entity/Q267282')]
[('Awards', '4'), ('ProductionCompanies', 'Columbia Pictures'), ('IRI', 'http://www.wikidata.org/entity/Q186941')]
[('Awards', '3'), ('ProductionCompanies', 'Film4 Productions'), ('IRI', 'http://www.wikidata.org/entity/Q5448886')]
[('Awards', '2'), ('ProductionCompanies', 'ITC Entertainment'), ('IRI', 'http://www.wikidata.org/entity/Q3149951')]
[('Awards', '2'), ('ProductionCompanies', 'Universal Pictures'), ('IRI',